# 导入环境

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('../../dataset/wanzi.json')
ds = Dataset.from_pandas(df)

In [3]:
ds[:3]

{'instruction': ['你好', '你好', '您好'],
 'input': ['', '', ''],
 'output': ['您好，我是 丸子，一个由 MEMECT 开发的 AI 助手，很高兴认识您。请问我能为您做些什么？',
  '您好，我是 丸子，一个由 MEMECT 打造的人工智能助手，请问有什么可以帮助您的吗？',
  '您好，我是 丸子，很高兴为您服务。有什么我可以帮您解决的问题或者需要我提供的帮助吗？']}

# 处理数据集

In [4]:
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-fs/glm-4-9b-chat/', use_fast=False, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [5]:
tokenizer.eos_token

'<|endoftext|>'

In [6]:
def process_func(example):
    MAX_LENGTH = 384
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer((f"[gMASK]<sop><|system|>\n假设你是 丸子，一个由 MEMECT 开发的 AI 助手。<|user|>\n"
                            f"{example['instruction']+example['input']}<|assistant|>\n"
                            ).strip(), 
                            add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 80
})

In [8]:
print(tokenizer.decode(tokenized_id[0]['input_ids']))
print(tokenized_id[0]['input_ids'])
print(tokenizer.decode([151331, 151333, 151335]))
print(tokenizer.encode('[gMASK]<sop><|system|>', add_special_tokens=False))

[gMASK] <sop> <|system|> 
假设你是 丸子，一个由 MEMECT 开发的 AI 助手。 <|user|> 
你好 <|assistant|> 您好，我是 丸子，一个由 MEMECT 开发的 AI 助手，很高兴认识您。请问我能为您做些什么？ <|endoftext|>
[151331, 151333, 151335, 198, 104536, 103408, 220, 116753, 3837, 98444, 98582, 24404, 5935, 103065, 103192, 15223, 54235, 102, 98520, 1773, 151336, 198, 109377, 151337, 121658, 3837, 101328, 220, 116753, 3837, 98444, 98582, 24404, 5935, 103065, 103192, 15223, 54235, 102, 98520, 3837, 118295, 100119, 99526, 1773, 98964, 106546, 98342, 107410, 98540, 110000, 11314, 151329]
[gMASK] <sop> <|system|>
[151331, 151333, 151335]


In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'您好，我是 丸子，一个由 MEMECT 打造的人工智能助手，请问有什么可以帮助您的吗？ <|endoftext|>'

# 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained('/root/autodl-fs/glm-4-9b-chat/', device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True)
model

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(151552, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-39): 40 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): SdpaAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [12]:
model.dtype

torch.bfloat16

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["query_key_value", "dense", "dense_h_to_4h", "dense_4h_to_h"],  # 现存问题只微调部分演示即可
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value', 'dense_4h_to_h', 'dense', 'dense_h_to_4h'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/root/autodl-fs/glm-4-9b-chat/', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'query_key_value', 'dense_4h_to_h', 'dense', 'dense_h_to_4h'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [15]:
model.print_trainable_parameters()

trainable params: 21,176,320 || all params: 9,421,127,680 || trainable%: 0.2248


# 配置训练参数

In [16]:
args = TrainingArguments(
    output_dir="/root/autodl-tmp/glm4-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    logging_steps=50,
    num_train_epochs=2,
    save_steps=100,
    learning_rate=1e-5,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


# 保存 LoRA 和 tokenizer 结果


In [19]:
peft_model_id="/root/autodl-tmp/glm4-lora/wanzi"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('/root/autodl-tmp/glm4-lora/wanzi/tokenizer_config.json',
 '/root/autodl-tmp/glm4-lora/wanzi/special_tokens_map.json',
 '/root/autodl-tmp/glm4-lora/wanzi/tokenizer.model',
 '/root/autodl-tmp/glm4-lora/wanzi/added_tokens.json')

# 加载 lora 权重推理

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/root/autodl-fs/glm-4-9b-chat/'
lora_path = '/root/autodl-tmp/glm4-lora/wanzi/'

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "user", "content": "假设你是 丸子，一个由 MEMECT 开发的 AI 助手。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.
/root/miniconda3/envs/py312/lib/python3.12/site-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)